In [1]:
import pandas as pd
import numpy as np
import sys
from os import path, makedirs

In [2]:
#Only want term 9 data
start_date = '2019-07-02'
end_date = '2024-07-16'
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)
raw_data_folder = "raw_data"

In [3]:
issues = pd.read_csv(path.join(raw_data_folder, "votes.csv"))
issues.loc[:, 'timestamp'] = pd.to_datetime(issues.timestamp.str[:10])
issues = issues[(issues.timestamp >= start_date) & (issues.timestamp <= end_date)]
issues = issues[issues['is_main'] == 1]
clean_data_folder = "main_data"
issues

,id,timestamp,display_title,reference,description,is_main,is_featured,procedure_reference,procedure_title
3,108532,2019-07-18 00:00:00,Resolution on the situation in Venezuela,RC-B9-0006/2019,Résolution,True,False,2019/2730(RSP),Resolution on the situation in Venezuela
15,108568,2019-09-18 00:00:00,Amending budget 2/2019: Horizon 2020 and Erasmus+,A9-0004/2019,Résolution,True,True,2019/2022(BUD),Amending budget 2/2019: Horizon 2020 and Erasmus+
23,108660,2019-09-18 00:00:00,Resolution on the state of play of the UK’s wi...,B9-0038/2019,Résolution,True,False,2019/2817(RSP),Resolution on the state of play of the UK’s wi...
28,108749,2019-09-19 00:00:00,Resolution on the state of implementation of t...,B9-0045/2019,Résolution,True,False,2019/2820(RSP),Resolution on the state of implementation of t...
31,108763,2019-09-19 00:00:00,"Resolution on Iran, notably the situation of w...",RC-B9-0089/2019,Résolution,True,False,2019/2823(RSP),"Resolution on Iran, notably the situation of w..."
...,...,...,...,...,...,...,...,...,...
19218,169157,2024-04-25 00:00:00,"Resolution on Azerbaijan, notably the repressi...",RC-B9-0227/2024,Proposition de résolution (ensemble du texte),True,False,2024/2698(RSP),"Resolution on Azerbaijan, notably the repressi..."
19236,169182,2024-04-25 00:00:00,Resolution on Iran’s unprecedented attack agai...,RC-B9-0235/2024,Proposition de résolution (ensemble du texte),True,False,2024/2704(RSP),Resolution on Iran’s unprecedented attack agai...
19241,169187,2024-04-25 00:00:00,Resolution on attempts to reintroduce a foreig...,RC-B9-0244/2024,Proposition de résolution (ensemble du texte),True,False,2024/2703(RSP),Resolution on attempts to reintroduce a foreig...
19250,169197,2024-04-25 00:00:00,Resolution on the proposed repeal of the law b...,RC-B9-0228/2024,Proposition de résolution (ensemble du texte),True,False,2024/2699(RSP),Resolution on the proposed repeal of the law b...


In [4]:
issues = issues[['id', 'display_title', 'reference', 'description', 'procedure_reference', 'procedure_title']]
titles, idx = np.unique(issues.display_title.astype(str), return_index=True)
issues = issues.iloc[idx]
vote_ids = issues[["id", "display_title"]]

In [5]:
# Make a matrix where the rows are votes and the columns are the MPs
member_votes = pd.read_csv(path.join(raw_data_folder, "member_votes.csv"))
useful_votes = member_votes[member_votes.vote_id.isin(vote_ids.id)]
member_country = useful_votes[["member_id", "country_code"]]
_, idx = np.unique(member_country.member_id, return_index=True)
member_country = member_country.iloc[idx]
member_country["ones"] = int(1)
member_country = member_country.pivot(index='member_id', columns='country_code', values='ones').fillna(0)
member_country.sort_index(inplace=True)
member_country = member_country.astype(int)
member_country.drop(columns=["GBR"], inplace=True)

try:
    member_country.to_csv(path.join(clean_data_folder, "member_country.csv"))
except OSError:
    makedirs(clean_data_folder)
    member_country.to_csv(path.join(clean_data_folder, "member_country.csv"))

# FOR
useful_votes = useful_votes[["member_id", "vote_id", "position"]]
useful_votes.loc[:, "position"] = useful_votes.position == "FOR"
useful_votes = useful_votes.pivot(index='member_id', columns='vote_id', values='position').fillna(False)
useful_votes.sort_index(inplace=True)
useful_votes.to_csv(path.join(clean_data_folder, "member_vote_for.csv"))

# AGAINST
useful_votes = member_votes[member_votes.vote_id.isin(vote_ids.id)]
useful_votes = useful_votes[["member_id", "vote_id", "position"]]
useful_votes.loc[:, "position"] = useful_votes.position == "AGAINST"
useful_votes = useful_votes.pivot(index='member_id', columns='vote_id', values='position').fillna(False)
useful_votes.sort_index(inplace=True)
useful_votes.to_csv(path.join(clean_data_folder, "member_vote_against.csv"))

/var/folders/p3/_v8g_c7n34g8knydcykyh7yr0000gn/T/ipykernel_14683/1009596238.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  useful_votes = useful_votes.pivot(index='member_id', columns='vote_id', values='position').fillna(False)
/var/folders/p3/_v8g_c7n34g8knydcykyh7yr0000gn/T/ipykernel_14683/1009596238.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  useful_votes = useful_votes.pivot(index='member_id', columns='vote_id', values='position').fillna(False)
